In [1]:
## Path for modules

import sys

import numpy as np
import xarray as xr

from netCDF4 import Dataset

import cartopy.crs as ccrs
import cartopy.feature as cfeature
from cartopy.mpl.gridliner import LONGITUDE_FORMATTER, LATITUDE_FORMATTER

import numpy.ma as ma

import matplotlib.cm as mplcm
import matplotlib.colors as colors
from mpl_toolkits.axes_grid1 import make_axes_locatable

seq_cmap = mplcm.Blues
div_cmap = mplcm.seismic

import matplotlib.pyplot as plt
import matplotlib as mpl
from datetime import date, datetime

import pandas as pd

%matplotlib inline

In [2]:
## Dataset

dirfilt_24h_ave = "/mnt/meom/workdir/henelle/eNATL60/JFM-JAS/"

In [3]:
## JAS files

tfileJAS_strain00 = dirfilt_24h_ave + 'eNATL60ACO-BLB002_y2009_JAS.1d_strain10m.nc'
tfileJAS_strainT0 = dirfilt_24h_ave + 'eNATL60ACO-BLBT02_y2009_JAS.1d_strain10m.nc'
tfileJAS_curl00   = dirfilt_24h_ave + 'eNATL60ACO-BLB002_y2009_JAS.1d_curl10m.nc'
tfileJAS_curlT0   = dirfilt_24h_ave + 'eNATL60ACO-BLBT02_y2009_JAS.1d_curl10m.nc'

In [4]:
## JFM files

tfileJFM_strain00 = dirfilt_24h_ave + 'eNATL60ACO-BLB002_y2010_JFM.1d_strain10m.nc'
tfileJFM_strainT0 = dirfilt_24h_ave + 'eNATL60ACO-BLBT02_y2010_JFM.1d_strain10m.nc'
tfileJFM_curl00   = dirfilt_24h_ave + 'eNATL60ACO-BLB002_y2010_JFM.1d_curl10m.nc'
tfileJFM_curlT0   = dirfilt_24h_ave + 'eNATL60ACO-BLBT02_y2010_JFM.1d_curl10m.nc'

In [5]:
# Curl files

fcurl_JAS_00 = xr.open_dataset(tfileJAS_curl00)
fcurl_JAS_T0 = xr.open_dataset(tfileJAS_curlT0)

fcurl_JFM_00 = xr.open_dataset(tfileJFM_curl00)
fcurl_JFM_T0 = xr.open_dataset(tfileJFM_curlT0)

In [6]:
# Strain files

fstrain_JAS_00 = xr.open_dataset(tfileJAS_strain00)
fstrain_JAS_T0 = xr.open_dataset(tfileJAS_strainT0)

fstrain_JFM_00 = xr.open_dataset(tfileJFM_strain00)
fstrain_JFM_T0 = xr.open_dataset(tfileJFM_strainT0)

In [7]:
navlat = fcurl_JAS_00['nav_lat']
navlon = fcurl_JAS_00['nav_lon']

In [8]:
## Curl

curl_JAS_00 = fcurl_JAS_00['socurloverf']
curl_JAS_T0 = fcurl_JAS_T0['socurloverf']

curl_JFM_00 = fcurl_JFM_00['socurloverf']
curl_JFM_T0 = fcurl_JFM_T0['socurloverf']

In [9]:
## Strain

strain_JAS_00 = fstrain_JAS_00['sostrainoverf']
strain_JAS_T0 = fstrain_JAS_T0['sostrainoverf']

strain_JFM_00 = fstrain_JFM_00['sostrainoverf']
strain_JFM_T0 = fstrain_JFM_T0['sostrainoverf']

In [10]:
print('min curl JAS 00 = ',np.min(curl_JAS_00))
print('max curl JAS 00 = ',np.max(curl_JAS_00))

min curl JAS 00 =  <xarray.DataArray 'socurloverf' ()>
array(-3.1714456, dtype=float32)
max curl JAS 00 =  <xarray.DataArray 'socurloverf' ()>
array(2.409554, dtype=float32)


In [11]:
def plot_surf(data1,data2,lon,lat,i,vmin,vmax,cmap,var_title,date,season,var):
    
    fig = plt.figure(figsize=(10,9))
    
    ax1 = fig.add_subplot(121,projection=ccrs.PlateCarree())
    ax2 = fig.add_subplot(122,projection=ccrs.PlateCarree())
    
    # Adjust subplots
    plt.subplots_adjust(hspace=0.0,wspace=0.0) # 0.025
    
    norm_fld = colors.Normalize(vmin=vmin, vmax=vmax, clip=False)
    
    pcolor1 = ax1.pcolormesh(lon,lat,data1,cmap=cmap,vmin=vmin,vmax=vmax,norm = norm_fld)
    pcolor2 = ax2.pcolormesh(lon,lat,data2,cmap=cmap,vmin=vmin,vmax=vmax,norm = norm_fld)
    
    # Add the Azores
    land = cfeature.GSHHSFeature(scale='intermediate',
                                 levels=[1],
                                 facecolor='dimgray')
    ax1.add_feature(land)
    ax2.add_feature(land)
    
    
    # Colorbar ax1
    cax1,kw1   = mpl.colorbar.make_axes(ax1,location='bottom',pad=0.05,shrink=0.8)
    out1       = fig.colorbar(pcolor1,cax=cax1,extend='both',**kw1)
    out1.set_label(var_title,size=14)
    out1.ax.tick_params(labelsize=14)
    xticks_ax1 = np.linspace(vmin,vmax,num=5)
    out1.set_ticks(xticks_ax1)
    
    # Colorbar ax2
    cax2,kw2   = mpl.colorbar.make_axes(ax2,location='bottom',pad=0.05,shrink=0.8)
    out2       = fig.colorbar(pcolor2,cax=cax2,extend='both',**kw2)
    out2.set_label(var_title,size=14)
    out2.ax.tick_params(labelsize=14)
    xticks_ax2 = np.linspace(vmin,vmax,num=5)
    out2.set_ticks(xticks_ax2)
    
    # Grid    
    gl1            = ax1.gridlines(crs=ccrs.PlateCarree(),draw_labels=True,lw=1,color='gray',alpha=0.8, ls='--')
    gl1.xformatter = LONGITUDE_FORMATTER
    gl1.yformatter = LATITUDE_FORMATTER
    gl1.xlabel_style = {'size': 10, 'color': 'dimgray'}
    gl1.ylabel_style = {'size': 10, 'color': 'dimgray'}
    gl1.xlabels_top = False
    gl1.ylabels_right = False
    gl2            = ax2.gridlines(crs=ccrs.PlateCarree(),draw_labels=True,lw=1,color='gray',alpha=0.8, ls='--')
    gl2.xformatter = LONGITUDE_FORMATTER
    gl2.yformatter = LATITUDE_FORMATTER
    gl2.xlabel_style = {'size': 10, 'color': 'dimgray'}
    gl2.ylabel_style = {'size': 10, 'color': 'dimgray'}
    gl2.xlabels_top = False
    gl2.ylabels_right = False
    gl2.ylabels_left = False
    
    # Adjusting axes
    ax1.set_xlim((-36, -26))
    ax1.set_ylim((25, 40))
    ax2.set_xlim((-36, -26))
    ax2.set_ylim((25, 40))

    ts = pd.to_datetime(str(date))
    d = ts.strftime('%d/%m/%y %H:%M')
    
    ax1.set_title(var_title+' 24h average no tide',size=14, y=1.05)
    ax2.set_title(var_title+' 24h average tide',size=14, y=1.05)
    plt.suptitle(var_title+' evolution '+season+'; '+str(d),size=22,y=1.0)
    
    plt.savefig('./animation_files_24h_ave/24h_ave_file_'+var+'_'+season+'_'+str(str(i).zfill(2))+'.png',dpi=100,
                bbox_inches='tight',pad_inches=0.1)
    plt.clf()
    

### JFM

In [12]:
now_JFM = datetime.now()
current_time_JFM = now_JFM.strftime("%H:%M:%S")
print("Current Time, before JFM =", current_time_JFM)

Current Time, before JFM = 16:34:36


In [13]:
#n = len(curl_JFM_00[:,0,0]) # number of time steps; curl
n = len(strain_JFM_00[:,0,0]) # number of time steps, strain
#n = 1

#### Curl

for i in range(n):
    plot_surf(curl_JFM_00[i,:,:],curl_JFM_T0[i,:,:],navlon,navlat,i,
              vmin=-0.3,vmax=0.3,cmap='RdYlBu_r',var_title='$\zeta/f$',
              date=str(fcurl_JFM_00.time_counter.values[i]),season='JFM',var='curl_cb03')

#### Strain

In [14]:
for i in range(n):
    plot_surf(strain_JFM_00[i,:,:],strain_JFM_T0[i,:,:],navlon,navlat,i,
              vmin=0,vmax=0.5,cmap='YlGn',var_title='s/f',
              date=str(fstrain_JFM_00.time_counter.values[i]),season='JFM',var='strain')

<ipython-input-11-fd9eb10fbb15>:3: RuntimeWarning: More than 20 figures have been opened. Figures created through the pyplot interface (`matplotlib.pyplot.figure`) are retained until explicitly closed and may consume too much memory. (To control this warning, see the rcParam `figure.max_open_warning`).
  fig = plt.figure(figsize=(10,9))


<Figure size 720x648 with 0 Axes>

<Figure size 720x648 with 0 Axes>

<Figure size 720x648 with 0 Axes>

<Figure size 720x648 with 0 Axes>

...

<Figure size 720x648 with 0 Axes>

<Figure size 720x648 with 0 Axes>

<Figure size 720x648 with 0 Axes>

<Figure size 720x648 with 0 Axes>

<Figure size 720x648 with 0 Axes>

### JAS

In [15]:
now_JAS = datetime.now()
current_time_JAS = now_JAS.strftime("%H:%M:%S")
print("Current Time, before JAS =", current_time_JAS)

Current Time, before JAS = 16:42:30


In [16]:
#n = len(curl_JAS_00[:,0,0]) # number of time steps; curl
n = len(strain_JAS_00[:,0,0]) # number of time steps, strain
#n = 1
#n = 45

#### Curl

for i in range(n):
    plot_surf(curl_JAS_00[i,:,:],curl_JAS_T0[i,:,:],navlon,navlat,i,
              vmin=-0.3,vmax=0.3,cmap='RdYlBu_r',var_title='$\zeta/f$',
              date=str(fcurl_JAS_00.time_counter.values[i]),season='JAS',var='curl_cb03')

#### Strain

In [17]:
for i in range(n):
    plot_surf(strain_JAS_00[i,:,:],strain_JAS_T0[i,:,:],navlon,navlat,i,
              vmin=0,vmax=0.5,cmap='YlGn',var_title='s/f',
              date=str(fstrain_JAS_00.time_counter.values[i]),season='JAS',var='strain')

<ipython-input-11-fd9eb10fbb15>:3: RuntimeWarning: More than 20 figures have been opened. Figures created through the pyplot interface (`matplotlib.pyplot.figure`) are retained until explicitly closed and may consume too much memory. (To control this warning, see the rcParam `figure.max_open_warning`).
  fig = plt.figure(figsize=(10,9))


<Figure size 720x648 with 0 Axes>

<Figure size 720x648 with 0 Axes>

<Figure size 720x648 with 0 Axes>

<Figure size 720x648 with 0 Axes>

...

<Figure size 720x648 with 0 Axes>

<Figure size 720x648 with 0 Axes>

<Figure size 720x648 with 0 Axes>

<Figure size 720x648 with 0 Axes>

<Figure size 720x648 with 0 Axes>

In [18]:
now = datetime.now()
current_time = now.strftime("%H:%M:%S")
print("Current Time, png files are done =", current_time)

Current Time, png files are done = 16:50:24
